In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
# from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from dataset import ImageDataset
import numpy as np
import torch
from ssim import ssim

DIV2KVal/DIV2K_valid_HR/


c:\Users\lubos\Documents\Projekty\m22\knn2\ssim.py:201: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [2]:
print(torch.cuda.is_available())

True


In [3]:
def compute_psnr(gt, pred):
    """Compute Peak Signal-to-Noise Ratio (PSNR)."""
    mse = np.mean((gt - pred) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0  # Pokud jsou obrázky normalizované (0,1), jinak použij 255.0
    return 10 * np.log10(max_pixel**2 / mse)

def compute_ssim(gt, pred):
    """Compute Structural Similarity Index (SSIM)."""
    # Convert numpy arrays to PyTorch tensors
    gt = torch.tensor(gt).permute(2, 0, 1).unsqueeze(0)  # (H, W, C) → (1, C, H, W)
    pred = torch.tensor(pred).permute(2, 0, 1).unsqueeze(0)  # (H, W, C) → (1, C, H, W)

    return ssim(
        gt, pred,
        window_size=11,
        in_channels=gt.shape[1],
        L=1  # Change to 255 if your images are in the 0-255 range
    )

In [4]:
# Load models and their respective checkpoints
def load_model(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model = checkpoint['model']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    return model

def evaluate_model(model, dataloader):
    model.eval()
    psnr_values = []
    ssim_values = []

    for lr, hr in dataloader:
        lr = lr.to(next(model.parameters()).device)  # Přenos na správné zařízení
        hr = hr.to(next(model.parameters()).device)

        with torch.no_grad():
            sr = model(lr)

        sr_np = sr.squeeze(0).permute(1, 2, 0).cpu().numpy()
        hr_np = hr.squeeze(0).permute(1, 2, 0).cpu().numpy()

        psnr_values.append(compute_psnr(hr_np, sr_np))
        ssim_values.append(compute_ssim(hr_np, sr_np))

    return np.mean(psnr_values), np.mean(ssim_values)

In [5]:
model_checkpoints = [
    "./base/8x256u_c5x4_c3x4.pth",
]

# Load dataset
dataset = ImageDataset(dataset_name="DIV2KVal", train=False, scale=8, downscale=1, crop=1024)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Run evaluation
results = {}
for checkpoint in model_checkpoints:
    model = load_model(checkpoint)
    mean_psnr, mean_ssim = evaluate_model(model, dataloader)
    results[checkpoint] = {"PSNR": mean_psnr, "SSIM": mean_ssim}
    print(f"{checkpoint}: PSNR={mean_psnr:.2f}, SSIM={mean_ssim:.4f}")

# Store results
with open("evaluation_results.txt", "w") as f:
    for model_name, metrics in results.items():
        f.write(f"{model_name}: PSNR={metrics['PSNR']:.2f}, SSIM={metrics['SSIM']:.4f}\n")

print("Evaluation complete. Results saved to evaluation_results.txt")

DIV2KVal/DIV2K_valid_HR/


KeyboardInterrupt: 